In [28]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

In [29]:
wiki = pd.read_csv('../data/wikipedia.csv')
uncy = pd.read_csv('../data/uncyclopedia.csv')

In [32]:
wiki.head(10)

,Field1
0,京都大学（きょうとだいがく、英語: Kyoto University）は、京都府京都市左京区...
1,京都大学は、日本で2番目に創設された帝国大学たる京都帝国大学の流れを汲んでいる国立大学で、入...
2,創立以来「自由の学風」を建学の精神としている。国立大学法人となった後に制定された京都大学の基...
3,京都市内の繁華街から離れた古都の風情を残す落ち着いた環境の中にあり、何事も学生の自主性に任せ...
4,京都大学や（前身の）京都帝国大学では、それらの自由の拡大あるいは縮小を巡る攻防戦が繰り返され...
5,京都大学は学部・研究科以外に19の附置研究所・研究センターを設置している[5]。2010年3...
6,タイムズ・ハイアー・エデュケーションの『THE世界大学ランキング 2021』（2020年）で...
7,2019年10月10日現在、卒業生から8人、ゆかりのある人物も含めると11人のノーベル賞受賞...
8,京都大学の起源は文久元年8月16日（1861年9月20日）に長崎に設立された長崎養生所（その...
9,帝国大学令制定後、近畿地方にも帝国大学の設置を求める声が上がっていたが、財政難のため見送られ...


In [33]:
uncy.head(10)

,Field1
0,京都大学（きょうとだいかく、英称：Kyoto University）は、京都府京都市を本拠地...
1,"「1人の天才と99,999人の狂人を生み出す大学」と呼ばれ、99,999人の中には態度がデカ..."
2,今日、カステラ医学の権威として知られる長崎大学医学部を本家と仰ぐ、ハラタイラ・烏賊学部門が新...
3,建学当初から自由を基調とした考えが根底にある。部落開放同盟と利権を欲しいままにし、京都市役所...
4,また、官僚養成のトンキン大学や実学を推奨する神戸大学に対して、ニート養成や空想科学、エロゲ制...
5,永遠の校長こと折田彦市尊師を称えた銅像があったが、うら若き京大生達から溢れ出る芸術の息吹が尊...
6,出町柳のよそもん下宿生はテレビ大阪が写らない。\n
7,牛車は大学側が用意するので、自前で購入する必要は無い。ただ、烏帽子や狩衣については入学前に購...
8,学内には、独立国家・吉田寮が存在する。クスノキ前で麻雀をやったり、炬燵で鍋をつついている連中...
9,お隣のソースの効いたたこ焼き大学の総長は京大卒の坂田利夫である。\n


In [34]:
# Remove new line character
def parse(x):
    x = re.sub("\n","", x)
    return x

In [35]:
wiki['Field1'] = wiki['Field1'].map(lambda x: parse(x))
uncy['Field1'] = wiki['Field1'].map(lambda x: parse(x))

In [36]:
wiki['label'] = 1
uncy['label'] = 0

In [37]:
data = pd.concat([wiki, uncy])

In [38]:
y, X = data['label'], data['Field1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [39]:
train = pd.DataFrame({'label': y_train, 'text': X_train})
test = pd.DataFrame({'label': y_test, 'text': X_test})

In [40]:
test.to_csv('../data/test_orig.txt', sep='\t', index=False, header=False)
train.to_csv('../data/train_orig.txt', sep='\t', index=False, header=False)